Library requirements to run the following code.
(we strongly recommend that you create a virtual environment before installing these
See https://docs.python.org/3/tutorial/venv.html  Also make sure you are using Python 3.7 or newer )

* numpy 
* Pillow
* torch

## Part 1: How slow is Python (for numeric computation) compared to C?

In [1]:
from importlib import reload
from typing import List, Tuple
import datetime as dt
import helpers
import random

reload( helpers )

<module 'helpers' from '/home/teo/Dokumente/Vectorized Operations Talk/helpers.py'>

In [6]:
def make_random_arrays( n: int ) -> Tuple[ List[float], List[float] ]:
    a = [ random.random() for _ in range(n)]
    b = [ random.random() for _ in range(n)]
    
    return a, b

In [7]:
N = 10_000_000
a, b = make_random_arrays(N)

In [15]:
type(a)

list

In [17]:
a[:10] # first 10 elements of list

[0.13938566860645718,
 0.3622965342202479,
 0.01604670924412188,
 0.14729400999200237,
 0.06929583023394081,
 0.6243616561469579,
 0.5521376291746255,
 0.6547843664688222,
 0.3178712716535689,
 0.3179277136813281]

We might never have 10_000_000 customers... 

But we do have tens of millions of web interaction events, for  instance.


In [18]:
def dot_product_v0(a: List[float], b: List[float]) -> float:
    n = len(a)
    
    result = 0.0 
    for i in range(n):
        result += a[i] * b[i]
    
    return result

In [19]:
%%question 1_v0

How long does the call `dot_product_v0( a, b )` take when N = 10 million?
~ 0.6 ms|~ 6 ms|~ 60 ms|~ 600ms|~ 6 s

In [20]:
%%time

# a, b = make_arrays_at_random(N)
dot_product_v0(a, b)

CPU times: user 646 ms, sys: 0 ns, total: 646 ms
Wall time: 645 ms


2501516.5638909573

In [18]:
def dot_product_v1(a: List[float], b: List[float]) -> float:
    n = len(a)
    
    result = 0.0 
    for elem_a, elem_b in zip(a, b):
        result += elem_a * elem_b
    
    return result

In [22]:
%%question 1_v1

How long does the call `dot_product_v1( a, b )` take when N = 10 million?
~ 0.4 ms|~ 4 ms|~ 40 ms|~ 400ms|~ 4 s

In [23]:
%%timeit -n1 -r10

dot_product_v1(a, b)

419 ms ± 5.52 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [24]:
%%timeit -n1 -r10

dot_product_v1(a, b)

446 ms ± 33 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [25]:
def dot_product_v2(a: List[float], b: List[float]) -> float:
    n = len(a)
    
    result = sum( elem_a * elem_b for elem_a, elem_b in zip(a, b) )
        
    return result

In [26]:
%%question 1_v2

How long does the call `dot_product_v2( a, b )` take when N = 10 million?
~ 0.4 ms|~ 4 ms|~ 40 ms|~ 400ms|~ 4 s

In [27]:
%%timeit -n1 -r10

dot_product_v0(a, b)

673 ms ± 21.3 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [28]:
%%timeit -n1 -r10

dot_product_v2(a, b)

484 ms ± 24.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


### Doing the same in Ruby

In [29]:
%%question 1_rb

If we write the analogous of `dot_product_v0` and `dot_product_v2` in ruby. How would they perform, compared to Python.
v0_rb > v0_py, v2_rb > v2_py|v0_rb > v0_py, v2_rb < v2_py|v0_rb < v0_py, v2_rb > v2_py|v0_rb < v0_py, v2_rb < v2_py


In [34]:
reload(helpers)

<module 'helpers' from '/home/teo/Dokumente/Vectorized Operations Talk/helpers.py'>

In [36]:
%%run_ruby_program

require "./helpers.rb"

N = 10_000_000
a = Array.new(N) { rand() }
b = Array.new(N) { rand() }

def dot_product_v0_rb(a, b)
    n = a.size
    
    result = 0.0 
    for i in (0...n)
        result += a[i] * b[i]
    end
    
    result
end


def dot_product_v2_rb(a, b)
    a.zip( b ).map { |x, y| x * y }.sum
    # a.zip(b).inject { |accum, pair| accum + pair[0] * pair[1] }
end

measure('dot_product_v0_rb', n=10) { dot_product_v0_rb(a, b) }
measure('dot_product_v2_rb', n=10) { dot_product_v2_rb(a, b) }

ruby 2.7.0p0 (2019-12-25 revision 647ee6f091) [x86_64-linux-gnu]
dot_product_v0_rb time taken: 517.092 ± 5.077 ms (10 runs)
dot_product_v2_rb time taken: 1537.388 ± 191.670 ms (10 runs)



### Trying the same in crystal-lang

Trying the same in [crystal-lang](https://www.crystal-lang.org) A language heavily inspired but ruby but compiled, not interpreted!

In [3]:
%%run_crystal_program

require "./helpers"

N = 10_000_000
a = Array.new(N) { rand() }
b = Array.new(N) { rand() }

def dot_product_v0_rb(a, b)
    n = a.size
    
    result = 0.0 
    i = 0
    
    while i < n  
        result += a[i] * b[i]
        i += 1  
    end
    
    result
end


def dot_product_v2_rb(a, b)
    a.zip( b ).map { |x, y| x * y }.sum  
end

measure("dot_product_v0_rb", n=10) { dot_product_v0_rb(a, b) }
measure("dot_product_v2_rb", n=10) { dot_product_v2_rb(a, b) }

dot_product_v0_rb time taken: 153.717 ± 4.477 ms (10 runs)
dot_product_v2_rb time taken: 421.642 ± 27.021 ms (10 runs)




### Doing the same in C 

In [4]:
%%run_c_code

#include "helpers.c"

double dot_product_v0( float* a, float* b, int n ) {
    double result = 0;
    
    for( int i = 0; i < n; ++i ) {
        result += a[i] * b[i];
    }    
    return result;
}

int main() {
    int N = 10000000;

    float *a = (float*) malloc(N * sizeof(float));
    float *b = (float*) malloc(N * sizeof(float));

    randomize_array( a, N );
    randomize_array( b, N );

    timeit( 10, dot_product_v0(a, b, N) );
        
    free( a );
    free( b );
}

Running c code...:
sample #  1: 24.312 ms
sample #  2: 24.330 ms
sample #  3: 24.416 ms
sample #  4: 23.944 ms
sample #  5: 23.900 ms
sample #  6: 24.003 ms
sample #  7: 24.345 ms
sample #  8: 23.874 ms
sample #  9: 23.954 ms
sample # 10: 24.762 ms

24.184 ms ± 0.278 ms per loop (mean ± std. dev. of 10 runs)


## 2. Making Python faster than C

In [5]:
a0 = [ 1.0, 2.0, 3.0 ]
print( type(a0) )

<class 'list'>


In [6]:
import numpy as np

N = 10_000_000
a = np.random.rand(N)
b = np.random.rand(N)

In [7]:
a

array([0.81751709, 0.01889141, 0.52258106, ..., 0.99837801, 0.96163608,
       0.26371896])

In [8]:
type(a)

numpy.ndarray

In [9]:
%%question 2_np

How long does computing the dot product via `np.dot(a, b)` take?
 1 ms|10 ms|25 ms|50 ms|>50 ms 

In [11]:
%%timeit -n1 -r100

np.dot( a, b )

13.2 ms ± 3.05 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


This is the first example of a **vectorized operation**. 

Simple definition of vectorized operation:

- It's an operation (i.e. function call) that does not involve loops in a high level interpreted language (such as Python, Ruby, R, PHP, Lua, etc...)
- All looping is done at the C/C++ level or lower...
- It uses compact an efficient data structures such as np.ndarray

### How is this even possible!?

1. `numpy` is implemented in C under the hood. So this is not really Python running...
2. `numpy`'s code is optimized internally using various advanced techniques, such as taking advantage of SIMD features of the underlying CPU.


# 2.1  Multidimensional arrays

Vectorized operations really shine when dealing with multidimensional arrays!

A matrix is an example of a multidimensional array. 

More precisely, **a matrix is a two dimensional or "2d"-array**.

Even more precisely, **a matrix is a rank-2 _tensor_** 

### Example: 
A matrix with the 10 characteristics of 10.000  real estates would be a 10.000-by-10 matrix (10 million entries in total)

Similarly a matrix with 10 search criteria for each of 1.000 customers would be a ....


In [12]:
real_estates = np.random.rand(10_000, 10)
print( real_estates, "\n" )
print( real_estates.shape )

[[0.47891014 0.00579048 0.8141111  ... 0.67161829 0.00510869 0.635186  ]
 [0.27048119 0.69130997 0.81096686 ... 0.34898346 0.0557773  0.77774732]
 [0.89601698 0.20633795 0.31719482 ... 0.38851082 0.14627792 0.27208681]
 ...
 [0.92808318 0.31427837 0.08322502 ... 0.35377213 0.01100655 0.24598937]
 [0.23687208 0.92404894 0.49105104 ... 0.17916575 0.37254507 0.50249517]
 [0.19076363 0.70146181 0.57608471 ... 0.58467713 0.40260967 0.76022052]] 

(10000, 10)


In [13]:
users = np.random.rand(1000, 10)
print( users, "\n" )
print( users.shape )

[[0.61181557 0.8907004  0.53532973 ... 0.43719749 0.94805528 0.78103089]
 [0.36413575 0.08440433 0.81038138 ... 0.07126302 0.83193529 0.05982337]
 [0.76512857 0.78785905 0.78299466 ... 0.88818751 0.4752882  0.17877301]
 ...
 [0.52184017 0.20906067 0.00141348 ... 0.98794332 0.34172932 0.53246752]
 [0.71878447 0.31441191 0.12907061 ... 0.05158623 0.53572919 0.53681659]
 [0.58985326 0.96208274 0.80535018 ... 0.44140785 0.8194333  0.20298018]] 

(1000, 10)


In [14]:
def compute_user_real_estate_match( users: np.ndarray, 
                                    real_estates: np.ndarray ) -> np.ndarray:
    
    n_users = users.shape[0]
    print( f"n_users = {n_users}")
    n_real_estates = real_estates.shape[0]
    print( f"n_real_estates = {n_real_estates}")
    
    result = np.zeros( (n_users, n_real_estates) )
    
    for user_idx in range( n_users ):
        for re_idx in range( n_real_estates ):
            result[ user_idx, re_idx ] = dot_product_v1( users[user_idx, :], 
                                                         real_estates[re_idx, :] )
    
    return result

In [15]:
%%question 2m1

How many entries does the `result` matrix have?
10⁵|10⁷|10⁸|10⁹|10¹⁰

In [16]:
%%question 2m2

How many operations operations on single number are done to compute it?
~ 10*10.000+10*1_000=110.000 |~ 10.000*1.000=10 MM|~2*10.000*1.000=20 MM|~2*10.000*1.000*10 = 200 MM

In [19]:
%%time
match_matrix = compute_user_real_estate_match(users, real_estates)

n_users = 1000
n_real_estates = 10000
CPU times: user 43.9 s, sys: 37.7 ms, total: 44 s
Wall time: 44 s


In [20]:
%%timeit -n1 -r100
match_matrix_np = np.dot( users, real_estates.transpose() )

69.7 ms ± 25.6 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [23]:
match_matrix[:3, :3]

array([[1.96480205, 3.00044595, 1.80624857],
       [1.20766574, 1.54381355, 1.00101797],
       [2.16676086, 3.03512547, 1.92771138]])

In [24]:
match_matrix_np = np.dot( users, real_estates.transpose() )
match_matrix_np[:3, :3]

array([[1.96480205, 3.00044595, 1.80624857],
       [1.20766574, 1.54381355, 1.00101797],
       [2.16676086, 3.03512547, 1.92771138]])

## Tensors

In the context of computing, **Tensor** is just a fancier, more precise and technical word for a "multidimensional array". 

The name is due to their origin in physics, in the context of the study of tension forces in elastic surfaces (2D) or solids (3D).

The name _Tensorflow_ (most popular deep learning framework) comes from tensor.

Tensors are better understood graphically:

<img src="./tensors_1d_2d_3d.png" />

Thus:

1. Tensor of rank-1 = vector = (linear) array
2. Tensor of rank-2 = matrix
3. Tensor of rank-3 = "cube"
4. Tensor of rank-4 = array of cubes
5. Tensor of rank-5 = matix of cubes 

etc...

## **Example of a rank-3 tensor**

In [25]:
from PIL import Image
image = Image.open("./tensors_1d_2d_3d.png")
tensor = np.array(image)

print( tensor.shape )

(713, 954, 3)


## Example of a rank-4 tensor

In [26]:
image1 = Image.open("./tensors_1d_2d_3d.png").resize( (300,300) )
image2 = Image.open("./book_cover.webp").resize( (300,300) )
np.array(image1).shape, np.array(image2).shape

((300, 300, 3), (300, 300, 3))

In [27]:
tensor_r4 = np.stack( [np.array(image1), np.array(image2)] )
tensor_r4.shape

(2, 300, 300, 3)

## 3. Extreme vectorization -  GPUs / TPUs

**Key insight:** most vectorized operations are highly parallelizable

E.g. we are multiplying `a[0] * b[0]`, `a[1] * b[1]`, `a[2] * b[2]` and these operations do no depend on each other.

Furthermore, we are doing the _same operation_, multiplication, we are just varying the _values_ being multiplied.

### Enter  SIMD architectures:

SIMD = Single Instruction Multiple Data

Many modern architectures have special CPU level instructions to stream-line these kinds of computations, effectively carrying several multiplications (e.g. 4) within a single clock cycle!

<table>
   <tr>
   <td> <img src="ordinary-cpu.png" width="400" height="400"/> </td>
    <td> <img src="simd-cpu.png" width="400" height="400"/> </td>
   </tr>
</table

## Enter GPUs

GPU = Graphics Processing Unit

* Again: single instruction multiple data
* High parallelization (256+ cores)
* Separate RAM

In [28]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print( f"device={device}" )

device=cuda


In [29]:
a = np.random.rand(10000, 10)
b = np.random.rand(1000, 10)

In [30]:
%%timeit -n1 -r10
c = np.dot( a, b.transpose() )

45.8 ms ± 2.43 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [31]:
a_gpu = torch.tensor(a).to( device )
bT_gpu = torch.tensor(b.transpose()).to( device )

In [32]:
%%timeit -n1 -r10
c_gpu = torch.matmul( a_gpu, bT_gpu )

The slowest run took 110.83 times longer than the fastest. This could mean that an intermediate result is being cached.
127 µs ± 331 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [33]:
c_gpu = torch.matmul( a_gpu, bT_gpu )
print( c_gpu.shape )

torch.Size([10000, 1000])


There is a catch... Moving data between main memory and gpu takes some time as well.

In [34]:
%%timeit -n1 -r10
c = c_gpu.to('cpu')

36 ms ± 1.67 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Appendix: Latencies of different operations


| Operation | Actual Time | Human Time | Human Action |
| --- | --- | --- | --- |
| 1 CPU Cycle | 0.3 ns |  1 s | Heart beat |
| L1 cache acces| 0.5 ns | 2 s | | 
| L2 cache access | 2.8 ns | 9 s | Breath twice
| L3 cache access  | 12.9 ns | 43 s | Lookup something in a book
| Main memory reference | 100 ns | 5 min | Call a friend and ask a question
| Compress 1K bytes with Zippy | 3000 ns = 3 µs | 2.5 hr | A long nap
| Send 2K bytes over 1 Gbps network | 20 µs | 16 hrs |
| SSD random read | 150 µs | 41 hours | > 1 day
| Read 1 MB sequentially from memory | 250 µs | 69 hrs | A long weekend
| Round trip within same datacenter | 500 µs | 138 hrs | Almost a week
| Read 1 MB sequentially from SSD* | 1 ms | 277 hrs | 11 days
| (rotational) disk seek  | 10 ms | 115 days | A season
| Read 1 MB sequentially from disk | 20 ms | 231 days | ~weekdays in a year
| Send packet CA->Netherlands->CA . | 150 ms | 4.7 years | Finish University Degree


## References

Wikipedia: https://en.wikipedia.org/wiki/Graphics_processing_unit

Crystal (programming language): https://crystal-lang.org

numpy (library numeric computation in Python): https://numpy.org/doc/stable/user/index.html

PyTorch (library for numeric compution using GPU): https://numpy.org/doc/stable/user/index.html
